# Multi-cell geolift analysis

In [ ]:
import pandas as pd

import causalpy as cp
from causalpy.data.simulate_data import (
    generate_multicell_geolift_data,
)

In [ ]:
%load_ext autoreload
%autoreload 2
pd.set_option("display.precision", 2)
seed = 42

---
## TEMP - generate data

This section will be removed once the synthetic data generation is implemented. Then we can just use `cp.load_data`.

In [ ]:


df, weightings_true = generate_multicell_geolift_data(treatment_time=70)

In [ ]:
df

In [ ]:
weightings_true

In [ ]:
df.to_csv("../../../causalpy/data/geolift_multi_cell.csv", index=False)

---

In [ ]:
df = (
    cp.load_data("geolift_multi_cell")
    # .assign(time=lambda x: pd.to_datetime(x["time"]))
    .set_index("x")
)

treatment_time = pd.to_datetime("2022-01-01")
df.head()

## Analysis 1 - individual cell analysis

The first analysis approach is to analyze each treated geo individually.

## Analysis 2 - aggregate cell analysis

The second analysis approach is to aggregate the treated geos and analyze them as a group.